# Build and cleanup catalog from exoplanet archive PSComp

#### Performs the following steps: 
1. Removes controversial planets, circumbinaries, and objects not in transiting/RV systems
2. Crossmatches the Gaia-Kepler sample (Berger+ 2020)
3. Enforces stellar parameter consistency in each system
4. Recomputues $r_p/R_{\star}$ and $N_{\rm pl}$ for each system
5. Eliminates low-multiplicity and transiting systems with unreliable data
6. Flags remaining high-priority targets with missing data in need of follow-up

In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import newton

from oviraptor.utils import *
from oviraptor.constants import *

In [2]:
#MAINPATH = 'C:/Users/djhoo/Documents/Oviraptor-master/'
MAINPATH = "/Users/research/projects/oviraptor/"

pscomposite_file = MAINPATH + "Catalogs/exoarchive_pscomposite_20210531.csv"
ps_rv_multi_file = MAINPATH + "Catalogs/exoarchive_ps_rv_multis_20210531.csv"

# Read in ExoArchive data

In [3]:
# here's the data
keys, vals = read_csv_file(pscomposite_file, k_index=99, v_index=100)

data = {}
for k in keys:
    data[k] = np.array(get_csv_data(k, keys, vals))
    

# grab a reference key
k0 = list(data.keys())[0]

print('total number of queried objects =', len(data[k0]))

total number of queried objects = 4389


In [4]:
for k in data.keys():
    print(k)

pl_name
hostname
gaia_id
sy_snum
sy_pnum
cb_flag
discoverymethod
disc_year
disc_refname
disc_facility
disc_telescope
disc_instrument
pl_controv_flag
pl_orbper
pl_orbpererr1
pl_orbpererr2
pl_orbperlim
pl_rade
pl_radeerr1
pl_radeerr2
pl_radelim
pl_radj
pl_radjerr1
pl_radjerr2
pl_radjlim
pl_bmasse
pl_bmasseerr1
pl_bmasseerr2
pl_bmasselim
pl_bmassj
pl_bmassjerr1
pl_bmassjerr2
pl_bmassjlim
pl_bmassprov
pl_orbeccen
pl_orbeccenerr1
pl_orbeccenerr2
pl_orbeccenlim
ttv_flag
pl_ratror
pl_ratrorerr1
pl_ratrorerr2
pl_ratrorlim
pl_rvamp
pl_rvamperr1
pl_rvamperr2
pl_rvamplim
st_spectype
st_teff
st_tefferr1
st_tefferr2
st_tefflim
st_rad
st_raderr1
st_raderr2
st_radlim
st_mass
st_masserr1
st_masserr2
st_masslim
st_met
st_meterr1
st_meterr2
st_metlim
st_metratio
st_logg
st_loggerr1
st_loggerr2
st_logglim
rastr
ra
decstr
dec
sy_dist
sy_disterr1
sy_disterr2
sy_plx
sy_plxerr1
sy_plxerr2
sy_vmag
sy_vmagerr1
sy_vmagerr2
sy_kmag
sy_kmagerr1
sy_kmagerr2
sy_gaiamag
sy_gaiamagerr1
sy_gaiamagerr2
sy_kepmag
sy_kep

# Remove unwanted objects

In [5]:
# filter detection methods
keep = (data["discoverymethod"] == "Transit") + \
       (data["discoverymethod"] == "Radial Velocity") + \
       (data["discoverymethod"] == "Transit Timing Variations")

for k in data.keys():
    data[k] = data[k][keep]

print("removed {0} objects due to non-relevant DETECTION METHOD".format(np.sum(~keep)))


# controversial flag
bad = data["pl_controv_flag"] == "1"

for k in data.keys():
    data[k] = data[k][~bad]

print("removed {0} objects flagged as CONTROVERSIAL".format(np.sum(bad)))


# circumbinary planets
bad = data["cb_flag"] == "1"

for k in data.keys():
    data[k] = data[k][~bad]

print("removed {0} objects in CIRCUMBINARY systems".format(np.sum(bad)))


print("\nafter cuts, {0} objects remain in {1} systems \n".format(len(data[k0]), len(np.unique(data["hostname"]))))

print("{0} TRANSITING planets".format(np.sum(data["discoverymethod"] == "Transit")))
print("{0} RADIAL VELOCITY planets".format(np.sum(data["discoverymethod"] == "Radial Velocity")))
print("{0} TTV planets".format(np.sum(data["discoverymethod"] == "Transit Timing Variations")))

removed 193 objects due to non-relevant DETECTION METHOD
removed 14 objects flagged as CONTROVERSIAL
removed 14 objects in CIRCUMBINARY systems

after cuts, 4168 objects remain in 3073 systems 

3314 TRANSITING planets
833 RADIAL VELOCITY planets
21 TTV planets


# Fix planet count

In [6]:
for i, starname in enumerate(np.unique(data["hostname"])):
    use = data["hostname"] == starname
    
    data["sy_pnum"][use] = np.sum(use)

# Read in Kepler names

In [7]:
kepnamepath = MAINPATH + "Catalogs/kepler_names.txt"

# read in the stellar output parameters
with open(kepnamepath, "r") as infile:
    raw_kepnames = []
    
    for i, line in enumerate(infile):
        raw_kepnames.append(line.split(","))
            
raw_kepnames = np.array(raw_kepnames)

# strip off trailing \newline commands
for i in range(len(raw_kepnames)):
    raw_kepnames[i,-1] = raw_kepnames[i,-1].strip("\n").strip("\ ")

In [8]:
kepnames = {}

for i, k in enumerate(raw_kepnames[0]):
    kepnames[k] = raw_kepnames[1:,i]

# Read in Gaia DR2

In [9]:
gaiapath = MAINPATH + "Catalogs/berger_2020_gaia_kepler_tab2_output.txt"

# read in the stellar output parameters
with open(gaiapath, "r") as infile:
    raw_gaia_data = []
    
    for i, line in enumerate(infile):
        raw_gaia_data.append(line.split("&"))
            
raw_gaia_data = np.array(raw_gaia_data)


# strip off trailing \newline commands
for i in range(len(raw_gaia_data)):
    raw_gaia_data[i,-1] = raw_gaia_data[i,-1].strip("\n").strip("\ ")
    
    
gaia_stars = {}

for i, k in enumerate(raw_gaia_data[0]):
    gaia_stars[k] = raw_gaia_data[1:,i]

In [10]:
gaiapath = MAINPATH + "Catalogs/berger_2020_gaia_kepler_planets.txt"

gaia_planets = {}

gaia_planets["KIC"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=0)
gaia_planets["radius"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=3)
gaia_planets["radius_err1"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=4)
gaia_planets["radius_err2"] = np.loadtxt(gaiapath, skiprows=32, dtype="str", usecols=5)

# Cross-match Kepler vs. Gaia and combine

In [11]:
# populate empty stellar reference column
data["st_refname"] = np.array([""]*len(data[k0]))

In [12]:
gaia_kic = np.asarray(gaia_stars["KIC"], dtype="int")

for i in range(len(data[k0])):
    hostname = data["hostname"][i]
    
    if hostname[:3] == "Kep":
        
        for j, kname in enumerate(kepnames["kepler_name"]):
            if kname[:-2] == hostname:
                kic = int(kepnames["kepid"][j])
        
        match = gaia_kic == kic
        
        if np.sum(match) == 1:
            data["st_refname"][i] = "Berger et al. 2020"
            
            data["st_teff"][i] = gaia_stars["iso_teff"][match][0]
            data["st_tefferr1"][i] = gaia_stars["iso_teff_err1"][match][0]
            data["st_tefferr1"][i] = gaia_stars["iso_teff_err2"][match][0]
            data["st_tefflim"][i]  = "0"
            
            data["st_rad"][i] = gaia_stars["iso_rad"][match][0]
            data["st_raderr1"][i] = gaia_stars["iso_rad_err1"][match][0]
            data["st_raderr1"][i] = gaia_stars["iso_rad_err2"][match][0]
            data["st_radlim"][i]  = "0"
            
            data["st_mass"][i] = gaia_stars["iso_mass"][match][0]
            data["st_masserr1"][i] = gaia_stars["iso_mass_err1"][match][0]
            data["st_masserr1"][i] = gaia_stars["iso_mass_err2"][match][0]
            data["st_masslim"][i]  = "0"
            
            data["st_met"][i] = gaia_stars["iso_feh"][match][0]
            data["st_meterr1"][i] = gaia_stars["iso_feh_err1"][match][0]
            data["st_meterr1"][i] = gaia_stars["iso_feh_err2"][match][0]
            data["st_metlim"][i]  = "0"
            data["st_metratio"][i]  = "[Fe/H]"
            
            data["st_logg"][i] = gaia_stars["iso_logg"][match][0]
            data["st_loggerr1"][i] = gaia_stars["iso_logg_err1"][match][0]
            data["st_loggerr1"][i] = gaia_stars["iso_logg_err2"][match][0]
            data["st_logglim"][i]  = "0"

In [13]:
gaia_kic = np.asarray(gaia_planets["KIC"], dtype="int")

count = 0

for i in range(len(data[k0])):
    hostname = data["hostname"][i]
    
    if hostname[:3] == "Kep":
        
        for j, kname in enumerate(kepnames["kepler_name"]):
            if kname[:-2] == hostname:
                kic = int(kepnames["kepid"][j])
        
        match = gaia_kic == kic
        
        if np.sum(match) == 1:
            data["pl_rade"][i] = gaia_planets["radius"][match][0]
            data["pl_radeerr1"][i] = gaia_planets["radius_err1"][match][0]
            data["pl_radeerr1"][i] = gaia_planets["radius_err2"][match][0]
            data["pl_radelim"][i]  = "0"
            
            RERJ = REARTH/RJUP
            
            data["pl_radj"][i] = str(np.round(float(gaia_planets["radius"][match][0])*RERJ,3))
            data["pl_radjerr1"][i] = str(np.round(float(gaia_planets["radius_err1"][match][0])*RERJ,3))
            data["pl_radjerr1"][i] = str(np.round(float(gaia_planets["radius_err2"][match][0])*RERJ,3))
            data["pl_radjlim"][i]  = "0"

# Enforce stellar parameter consistency for each system

In [14]:
for i, star in enumerate(np.unique(data["hostname"])):
    use = data["hostname"] == star
    npl = int(data["sy_pnum"][use][0])
    
    for k in data.keys():
        if (k[:2] == "st")*(npl > 1):
            if np.any(data[k][use] != data[k][use][0]):
                
                # grab relevant values
                vals = data[k][use]
                
                # overwrite missing values
                if npl == 2:
                    if np.any(vals == ""):
                        data[k][use] = vals[vals != ""]
                
                
                # for high-multiplicity systems, use the most common value
                if npl > 2:
                    count = np.array([list(vals).count(v) for v in vals])
                    count[vals == ""] = 0
                    
                    if np.any(count > 0) * ~np.all(count == 1):
                        data[k][use] = data[k][use][np.argmax(count)]
                    
                
                
                # now enforce consistency by taking mean
                vals[vals == ""] = "nan"
                
                try:
                    data[k][use] = np.nanmean(np.asarray(vals, dtype="float"))
                except:
                    pass

# Check for missing stellar data (should be very few systems)

In [15]:
print("\n\nMissing MASS")
print(np.unique(data["hostname"][data["st_mass"] == ""]))

print("\n\nMissing RADIUS")
print(np.unique(data["hostname"][data["st_rad"] == ""]))

print("\n\nMissing TEMPERATURE")
print(np.unique(data["hostname"][data["st_teff"] == ""]))



Missing MASS
[]


Missing RADIUS
['GJ 667 C']


Missing TEMPERATURE
['SWEEPS-11' 'SWEEPS-4']


In [16]:
bad = (data["st_mass"] == "") + (data["st_rad"] == "") + (data["st_teff"] == "")
nsys = len(np.unique(data["hostname"][bad]))

for k in data.keys():
    data[k] = data[k][~bad]
    

print("Removed {0} systems missing stellar mass, radius, or temperature".format(nsys))

Removed 3 systems missing stellar mass, radius, or temperature


# Track non-*Kepler* stars in TICv8 compatible format

In [17]:
# make a list of non-Kepler stars to upload to MAST in order to query TICv8
# https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html

use = data["disc_facility"] != "Kepler"

nonkep_fmt = ["#@string", "ra", "dec"]
nonkep_header = ["TARGET", "RA", "DEC"]
nonkep_data = np.vstack([np.unique(data["hostname"][use]),
                         np.unique(data["ra"][use]),
                         np.unique(data["dec"][use])]).swapaxes(0,1)

nonkep_targets = np.vstack([nonkep_fmt, nonkep_header, nonkep_data])

np.savetxt("Catalogs/non_kepler_stars.csv", nonkep_targets, fmt=("%s", "%s", "%s"), delimiter=",")

# Label each system as "Transit", "Radial Velocity", or "Mixed"

In [18]:
data["system_type"] = np.array([""]*len(data[k0]), dtype="<U29")


for i, star in enumerate(data["hostname"]):
    use = data["hostname"] == star
    
    if np.all(data["discoverymethod"][use] == "Transit"):
        data["system_type"][use] = "Transit"
        
    elif np.all(data["discoverymethod"][use] == "Radial Velocity"):
        data["system_type"][use] = "Radial Velocity"
        
    else:
        data["system_type"][use] = "Mixed"

# Check for transiting planets missing $r_p/R_{\star}$ and calculate manually

In [19]:
bad = np.zeros(len(data["discoverymethod"]), dtype="bool")

check_keys = ["pl_ratror", "pl_ratrorerr1", "pl_ratrorerr2"]

for k in check_keys:
    bad += data[k] == ""
    bad += data[k] == "nan"
    
bad *= (data["discoverymethod"] == "Transit")

In [20]:
rad_keys = ["pl_rade", "pl_radeerr1", "pl_radeerr2", "st_rad", "st_raderr1", "st_raderr2"]
rad_data = {}

for i, k in enumerate(rad_keys):
    rad_data[k] = data[k][bad]
    rad_data[k][rad_data[k] == ""] = "nan"
    rad_data[k] = np.asarray(rad_data[k], dtype="float")
    
    
data["pl_ratror"][bad] = rad_data["pl_rade"]/rad_data["st_rad"]*REARTH/RSUN
data["pl_ratrorerr1"][bad] = rad_data["pl_radeerr1"]/rad_data["st_rad"]*REARTH/RSUN
data["pl_ratrorerr2"][bad] = rad_data["pl_radeerr2"]/rad_data["st_rad"]*REARTH/RSUN

In [21]:
# Remove any transiting planets still missing rp/Rs
bad = np.zeros(len(data["discoverymethod"]), dtype="bool")

check_keys = ["pl_ratror", "pl_ratrorerr1", "pl_ratrorerr2"]

for k in check_keys:
    bad += data[k] == ""
    bad += data[k] == "nan"
    
bad *= (data["discoverymethod"] == "Transit")

print("Removed the following transiting planets due to missing radius ratio information:")
print(data["pl_name"][bad])

for k in data.keys():
    data[k] = data[k][~bad]

Removed the following transiting planets due to missing radius ratio information:
['HD 3167 d' 'HD 72490 b' 'KOI-142 c' 'Kepler-430 b' 'Kepler-430 c'
 'Kepler-431 b' 'Kepler-431 c' 'Kepler-431 d']


# Check for RV planets missing RV semi-amplitude and calculate manually

In [22]:
bad = np.zeros(len(data["discoverymethod"]), dtype="bool")

check_keys = ["pl_rvamp", "pl_rvamperr1", "pl_rvamperr2"]

for k in check_keys:
    bad += data[k] == ""
    bad += data[k] == "nan"
    
bad *= (data["discoverymethod"] == "Radial Velocity")

In [23]:
mass_keys = ["pl_bmasse", "pl_bmasseerr1", "pl_bmasseerr2", "st_mass", "st_masserr1", "st_masserr2"]
mass_data = {}

for i, k in enumerate(mass_keys):
    mass_data[k] = data[k][bad]
    mass_data[k][mass_data[k] == ""] = "nan"
    mass_data[k] = np.asarray(mass_data[k], dtype="float")
    
    
data["pl_rvamp"][bad] = mass_data["pl_bmasse"]/mass_data["st_mass"]*MEARTH/MSUN
data["pl_rvamperr1"][bad] = mass_data["pl_bmasseerr1"]/mass_data["st_mass"]*MEARTH/MSUN
data["pl_rvamperr2"][bad] = mass_data["pl_bmasseerr2"]/mass_data["st_mass"]*MEARTH/MSUN

In [24]:
# Remove any radial velocity planets still missing RV semi-amplitude
bad = np.zeros(len(data["discoverymethod"]), dtype="bool")

check_keys = ["pl_rvamp", "pl_rvamperr1", "pl_rvamperr2"]

for k in check_keys:
    bad += data[k] == ""
    bad += data[k] == "nan"
    
bad *= (data["discoverymethod"] == "Radial Velocity")

print("Removed the following RV planets due to missing RV semi-amplitude information:")
print(data["pl_name"][bad])

for k in data.keys():
    data[k] = data[k][~bad]

Removed the following RV planets due to missing RV semi-amplitude information:
['91 Aqr b' 'HIP 63242 b' 'Kepler-93 c' 'Kepler-97 c' 'bet Pic c'
 'tau Gem b']


# Check that TTV planets have either a mass or radius with uncertainties

In [25]:
no_mass = np.zeros(len(data["discoverymethod"]), dtype="bool")
mkeys = ["pl_bmasse", "pl_bmasseerr1", "pl_bmasseerr2"]
for k in mkeys:
    no_mass += data[k] == ""
    no_mass += data[k] == "nan"
    
    
no_rad = np.zeros(len(data["discoverymethod"]), dtype="bool")
rkeys = ["pl_rade", "pl_radeerr1", "pl_radeerr2"]
for k in rkeys:
    no_rad += data[k] == ""
    no_rad += data[k] == "nan"

bad = (data["discoverymethod"] == "Transit Timing Variations")*no_mass*no_rad


print("Removed the following TTV planets missing both mass and radius informatino:")
print(data["pl_name"][bad])

for k in data.keys():
    data[k] = data[k][~bad]

Removed the following TTV planets missing both mass and radius informatino:
['Kepler-160 d']


# Check for targets missing uncertainties on key parameters

In [26]:
# Check for any stars missing uncertainties on stellar mass or radius
bad_mass = (data["st_masserr1"] == "") + (data["st_masserr2"] == "")
nbm = len(np.unique(data["hostname"][bad_mass]))

print("{0} stars are missing uncertainties on stellar MASS".format(nbm))


# Check for any stars missing uncertainties on stellar mass or radius
bad_radius = (data["st_raderr1"] == "") + (data["st_raderr2"] == "")
nbr = len(np.unique(data["hostname"][bad_radius]))

print("{0} stars are missing uncertainties on stellar RADIUS".format(nbr))


# Check for any stars missing uncertainties on stellar mass or radius
bad_temp = (data["st_tefferr1"] == "") + (data["st_tefferr2"] == "")
nbt = len(np.unique(data["hostname"][bad_temp]))

print("{0} stars are missing uncertainties on stellar TEMPERATURE".format(nbt))

180 stars are missing uncertainties on stellar MASS
120 stars are missing uncertainties on stellar RADIUS
129 stars are missing uncertainties on stellar TEMPERATURE


In [27]:
# flag high-multiplicity RV systems which need to be fixed manually
npl = np.array(data["sy_pnum"], dtype="int")
bad = (bad_mass + bad_radius + bad_temp)*(data["system_type"] == "Radial Velocity")*(npl > 2)


print("The following important RV systems are missing critical data:")
print(np.unique(data["hostname"][bad]))

The following important RV systems are missing critical data:
['GJ 180' 'GJ 3138' 'GJ 3293' 'GJ 433' 'GJ 876' 'HD 125612' 'HD 160691'
 'HD 20781' 'HD 27894' 'HD 31527' 'HD 37124' 'HIP 14810' 'Wolf 1061'
 'ups And']


In [28]:
# eliminate the low multiplicity and non-RV systems with missing data
bad = (bad_mass + bad_radius + bad_temp)*((data["system_type"] != "Radial Velocity") + (npl < 3))

for k in data.keys():
    data[k] = data[k][~bad]

print("Eliminated {0} planets from systems with incomplete stellar data".format(np.sum(bad)))

Eliminated 246 planets from systems with incomplete stellar data


# Fix planet count

In [29]:
for i, starname in enumerate(np.unique(data["hostname"])):
    use = data["hostname"] == starname
    
    data["sy_pnum"][use] = np.sum(use)

# Print out the list of high-multiplicity RV systems

In [30]:
npl = np.array(data["sy_pnum"], dtype="int")
use = (npl > 2)*(data["system_type"] == "Radial Velocity")

rv_hostname = np.unique(data["hostname"][use])

print("The following {0} RV systems host at least three planets:\n".format(len(rv_hostname)))
print(rv_hostname)

The following 38 RV systems host at least three planets:

['47 UMa' '55 Cnc' '61 Vir' 'DMPP-1' 'GJ 1061' 'GJ 163' 'GJ 180' 'GJ 3138'
 'GJ 3293' 'GJ 433' 'GJ 581' 'GJ 676 A' 'GJ 876' 'HD 10180' 'HD 125612'
 'HD 136352' 'HD 141399' 'HD 158259' 'HD 160691' 'HD 164922' 'HD 181433'
 'HD 20781' 'HD 20794' 'HD 215152' 'HD 219134' 'HD 27894' 'HD 31527'
 'HD 34445' 'HD 37124' 'HD 40307' 'HD 69830' 'HD 7924' 'HIP 14810'
 'HIP 57274' 'Wolf 1061' 'YZ Cet' 'tau Cet' 'ups And']


In [31]:
rv_ra = []
rv_dec = []

for i, rvhn in enumerate(rv_hostname):
    use = data["hostname"] == rvhn
    
    rv_ra.append(data["ra"][use][0])
    rv_dec.append(data["dec"][use][0])

In [32]:
# make a list of high-multiplicity RV stars to upload to MAST in order to query TICv8
# https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html

rv_fmt = ["#@string", "ra", "dec"]
rv_head = ["TARGET", "RA", "DEC"]
rv_data = np.vstack([rv_hostname, rv_ra, rv_dec]).swapaxes(0,1)

rv_mast_targets = np.vstack([rv_fmt, rv_head, rv_data])

np.savetxt("Catalogs/rv_multis_for_mast.csv", rv_mast_targets, fmt=("%s", "%s", "%s"), delimiter=",")

# Check RV stellar property sources

In [33]:
# here's the data
rv_keys, rv_vals = read_csv_file(ps_rv_multi_file, k_index=0, v_index=1)


rv_data = {}
for k in rv_keys:
    rv_data[k] = np.array(get_csv_data(k, rv_keys, rv_vals))

In [34]:
for i, star in enumerate(np.unique(rv_hostname)):
    use = rv_data["hostname"] == star
    
    refs = np.unique(rv_data["st_refname"][use])
    
    print("")
    print(star)
    print("-----------")
    for r in refs:
        
        
        loc = r.find("target=ref>")
        ref = r[loc+11:-4]
        
        if ref[0] == " ":
            ref = ref[1:]
        
        print("", ref)


47 UMa
-----------
 Naef et al. 2004 
 TICv8
 Wittenmyer et al. 2007 
 Wittenmyer et al. 2009 

55 Cnc
-----------
 Baluev 2015
 Bourrier et al. 2018
 Bourrier &amp; H&eacute;brard 2014
 Butler et al. 1997 
 Dai et al. 2019
 Demory et al. 2011
 Marcy et al. 2002 
 McArthur et al. 2004 
 Naef et al. 2004 
 Salz et al. 2015
 TICv8
 Winn et al. 2011 

61 Vir
-----------
 Vogt et al. 2010 

DMPP-1
-----------
 Staab et al. 2020

GJ 1061
-----------
 Dreizler et al. 2020

GJ 163
-----------
 Bonfils et al. 2013 

GJ 180
-----------
 Feng et al. 2020
 Tuomi et al. 2014

GJ 3138
-----------
 Astudillo-Defru et al. 2017

GJ 3293
-----------
 Astudillo-Defru et al. 2015
 Astudillo-Defru et al. 2017

GJ 433
-----------
 Delfosse et al. 2013 
 Feng et al. 2020
 Stassun et al. 2017
 Tuomi et al. 2014

GJ 581
-----------
 Bonfils et al. 2005 
 Mayor et al. 2009 
 TICv8
 Vogt et al. 2010

GJ 676 A
-----------
 Anglada-Escude & Tuomi 2012 
 Forveille et al. 2011 
 Sahlmann et al. 2016
 Stassun et al

# Write out catalog

In [35]:
WRITENEW = True
if WRITENEW:
    filepath = MAINPATH + 'Catalogs/oviraptor_A_cleaned_and_crossmatched.csv'

    with open(filepath, "w") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(data.keys())
        writer.writerows(zip(*data.values()))

print("Writing complete!")

Writing complete!
